# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [55]:
import main
# Zadanie 1
length = 100
df = main.pd.read_sql_query("""
SELECT title
FROM film
WHERE
    film.length = %s
""", con=main.connection, params=[length])
df 

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title
0,Behavior Runaway
1,Bilko Anonymous
2,Deliverance Mulholland
3,Divine Resurrection
4,Flatliners Killer
5,Gun Bonnie
6,Happiness United
7,Hyde Doctor
8,Pirates Roxanne
9,Punk Divorce


In [58]:
# Zadanie 2
city = "London"
df = main.pd.read_sql_query("""
SELECT first_name, last_name
FROM customer

JOIN address
ON address.address_id = customer.address_id

JOIN CITY
ON city.city_id = address.city_id

WHERE
    city = %s
""", con=main.connection, params=[city])
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name
0,Mattie,Hoffman
1,Cecil,Vines


In [41]:
# Zadanie 3
df = main.pd.read_sql_query("""
    SELECT
        AVG(payment.amount) AS avg
    FROM category

    INNER JOIN film_category
    ON film_category.category_id = category.category_id

    INNER JOIN film
    ON film.film_id = film_category.film_id

    INNER JOIN inventory
    ON inventory.film_id = film.film_id

    INNER JOIN rental
    ON rental.inventory_id = inventory.inventory_id

    INNER JOIN payment
    ON payment.rental_id = rental.rental_id

""", con=main.connection)

# Albo
df1 = main.pd.read_sql_query("""
SELECT AVG(payment.amount) AS avg
FROM payment
""", con=main.connection)

df1

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg
0,4.200606


In [42]:
# Zadanie 4
df = main.pd.read_sql_query("""
SELECT name, COUNT(film_category.category_id)
FROM category

JOIN film_category
ON film_category.category_id = category.category_id 

GROUP BY
    name
""", con=main.connection)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,count
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


In [43]:
# Zadanie 5
df = main.pd.read_sql_query("""
SELECT country.country, COUNT(*)
FROM customer

INNER JOIN address
ON address.address_id = customer.address_id

INNER JOIN city
ON city.city_id = address.city_id

INNER JOIN country
ON country.country_id = city.country_id

GROUP BY
    country.country
""", con=main.connection)

df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country,count
0,Bangladesh,3
1,Indonesia,14
2,Venezuela,7
3,Cameroon,2
4,Czech Republic,1
...,...,...
103,Tanzania,3
104,Poland,8
105,Greenland,1
106,French Guiana,1


In [112]:
# Zadanie 6
df = main.pd.read_sql_query("""
SELECT store.*, COUNT(address.address_id)
FROM address

JOIN customer
ON customer.address_id = address.address_id

JOIN store
ON store.address_id = address.address_id

GROUP BY store.store_id

HAVING COUNT(address.address_id) > %s AND COUNT(address.address_id) < %s
""", con=main.connection, params=[100, 300])

df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,store_id,manager_staff_id,address_id,last_update,count


In [50]:
# Zadanie 7
df = main.pd.read_sql_query("""
SELECT
    customer.first_name AS first_name,
    customer.last_name AS last_name,
    SUM(length) AS sum
FROM film

JOIN inventory
ON inventory.film_id = film.film_id

JOIN rental
ON rental.inventory_id = inventory.inventory_id

JOIN customer
ON customer.customer_id = rental.customer_id

GROUP BY
    customer.first_name, customer.last_name

HAVING SUM(length) >= %s

ORDER BY sum, customer.last_name, customer.first_name
""", con=main.connection, params=[200*60])
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,sum


In [52]:
# Zadanie 8
film_name = "Amadeus Holy"

df = main.pd.read_sql_query("""
SELECT title, AVG(payment.amount)
FROM film

JOIN inventory
ON inventory.film_id = film.film_id

JOIN rental
ON rental.inventory_id = inventory.inventory_id

JOIN payment
ON payment.rental_id = rental.rental_id

GROUP BY title
""", con=main.connection)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,avg
0,Graceland Dynamite,6.323333
1,Opus Ice,6.090000
2,Braveheart Human,3.390000
3,Wonderful Drop,5.865000
4,Rush Goodfellas,2.918571
...,...,...
953,Mockingbird Hollywood,2.444545
954,Gathering Calendar,1.990000
955,Drums Dynamite,1.913077
956,Samurai Lion,4.240000


In [59]:
# Zadanie 9
df = main.pd.read_sql_query("""
SELECT name, AVG(film.length)
FROM category

JOIN film_category
ON film_category.category_id = category.category_id

JOIN film
ON film.film_id = film_category.film_id

GROUP BY
name
""", con=main.connection)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


In [77]:
# Zadanie 10
df = main.pd.read_sql_query("""
SELECT name, film.title, film.length
FROM category

JOIN film_category
ON film_category.category_id = category.category_id

JOIN film
ON film.film_id = film_category.film_id

WHERE film.length = (
    SELECT MAX(film.length)
    FROM film
)
""", con=main.connection)
df


/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,title,length
0,Games,Chicago North,185
1,Comedy,Control Anthem,185
2,Action,Darn Forrester,185
3,Animation,Gangs Pride,185
4,Music,Home Pity,185
5,Travel,Muscle Bright,185
6,Animation,Pond Seattle,185
7,Sci-Fi,Soldiers Evolution,185
8,Travel,Sweet Brotherhood,185
9,Action,Worst Banger,185


In [76]:
# Zadanie 11
df = main.pd.read_sql_query("""
SELECT film.length, film.title
FROM category

JOIN film_category
ON film_category.category_id = category.category_id

JOIN film
ON film.film_id = film_category.film_id

WHERE film.length = (
    SELECT MAX(film.length)
    FROM film
)
""", con=main.connection)
df
# Jak widzimy najdluzszymi filmami sa te ktore maja po 185 min, jest ich az 10.
# Widzimy, ze output w porownaniu do zadania 10 jest ten sam.

,length,title
0,185,Chicago North
1,185,Control Anthem
2,185,Darn Forrester
3,185,Gangs Pride
4,185,Home Pity
5,185,Muscle Bright
6,185,Pond Seattle
7,185,Soldiers Evolution
8,185,Sweet Brotherhood
9,185,Worst Banger


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [30]:
import main

# Funkcja film_in_category
df = main.film_in_category(2)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,languge,category
0,Alter Victory,English,Animation
1,Anaconda Confessions,English,Animation
2,Argonauts Town,English,Animation
3,Bikini Borrowers,English,Animation
4,Blackout Private,English,Animation
...,...,...,...
61,Tracy Cider,English,Animation
62,Turn Star,English,Animation
63,Wait Cider,English,Animation
64,Watch Tracy,English,Animation


In [31]:
# Funkcja number_films_in_category
df = main.number_films_in_category(2)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,count
0,Animation,66


In [32]:
# Funkcja number_film_by_length
df = main.number_film_by_length(150, 200)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,length,count
0,150,8
1,151,7
2,152,9
3,153,9
4,154,7
5,155,6
6,156,4
7,157,6
8,158,6
9,159,6


In [33]:
# Funkcja client_from_city
df = main.client_from_city("Abu Dhabi")
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city,first_name,last_name
0,Abu Dhabi,Tom,Milner


In [34]:
# Funkcja avg_amount_by_length
df = main.avg_amount_by_length(100)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,length,avg
0,100,4.573214


In [36]:
# Funkcja client_by_sum_length
df = main.client_by_sum_length(1500)
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name,last_name,sum
0,Leona,Obrien,1588
1,Katherine,Rivera,1615
2,Tiffany,Jordan,1667
3,Jerome,Kenyon,1679
4,Penny,Neal,1738
...,...,...,...
592,Wesley,Bull,4808
593,Clara,Shaw,4808
594,Tammy,Sanders,5065
595,Eleanor,Hunt,5360


In [37]:
# Funkcja category_statistic_length
df = main.category_statistic_length("Action")
df

/home/piotr/anaconda3/envs/piotrWolnikEnv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
